# Create a Grafana Data Source and Dashboard

Follow the steps in this this tutorial to create a "stocks" Grafana dashboard using Iguazio data source <br>
Note that you need to create a Grafana service before running the notebook <br>
Creating a grafana service can be done from the "services" screen in the Platform

In [9]:
import nuclio

## Environment

In [31]:
%nuclio cmd python -m pip install git+https://github.com/v3io/grafwiz

  Cloning https://github.com/v3io/grafwiz to /tmp/pip-req-build-omi1hw7m
  Running setup.py bdist_wheel for grafwiz ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-r6t4pyvs/wheels/b2/74/24/52567131204eb68e9fb5cb64de7b99859c363820e3b0c68f65
Successfully built grafwiz


## Function

In [1]:
# nuclio: start-code

In [10]:
from grafwiz import *
import os

In [11]:
def deploy_dashboard(context, 
                     grafana_url:str = 'http://grafana', 
                     streamview_url:str = 'http://3.12.231.36:30100',
                     v3io_container:str = 'bigdata',
                     stocks_kv_table:str = 'stocks/stocks_kv',
                     stocks_tsdb_table:str = 'stocks/stocks_tsdb',
                     stocks_sentiment_tsdb_table:str = 'stocks/stocks_sentiment_tsdb'):
    # Create a new Grafana dashboard
    DataSource(name='stream-viewer', frames_url=streamview_url).deploy(grafana_url, use_auth=False)
    
    dash = Dashboard("stocks-test", start='now-1d', dataSource='Iguazio')

    # Add a symbol combo box (template) with data from the stocks table
    dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=stocks/stocks_kv;backend=kv;container=bigdata")

    # Create a table and log viewer in one row
    tbl = Table('tbl1',span=8).source(table=stocks_kv_table,fields=['symbol','volume', 'price','last_updated'],container=v3io_container)
    log = Table('log', dataSource='stream-viewer', span=4)
    dash.row([tbl,log])

    # Create 2 charts on the second row
    dash.row([Graph(metric).series(table=stocks_tsdb_table, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']])
    dash.row([Graph('sentiment').series(table=stocks_sentiment_tsdb_table, fields=['sentiment'], filter='symbol=="$SYMBOL"', container=v3io_container)])
    
    # Deploy to Grafana
    return dash.deploy(grafana_url)

In [12]:
# nuclio: end-code

## Local test

In [13]:
from mlrun import code_to_function, run_local

fn = code_to_function('grafana_dashboard_deployer',
                      kind='job',
                      handler='deploy_dashboard',
                      project='stocks')
fn.spec.build.base_image = 'mlrun/mlrun'
fn.export('06-grafana.yaml')

> 2020-09-27 11:04:42,807 [info] function spec saved to path: 06-grafana.yaml


In [ ]:
run_local(handler=deploy_dashboard)